In [34]:
###Solving Task Two od Project 2 ###
import numpy as np
import itertools

def Newton_system(F, DF, x0, N, eps):
    """
    Solve F(x) = 0 using Newton's method with a pseudo-inverse to handle singular 
    Jacobians that occurred when calculated not using a pseudo-inverse 
    
    Inputs:
      F: Function returning the residual vector F(x)
      DF: Function returning the Jacobian matrix DF(x)
      x0: Initial guess (vector)
      N: Maximum number of iterations
      eps: Convergence tolerance (stop when ||F(x)||_2 < eps)
    
    Output:
      x: Approximate solution vector
      steps: Number of steps taken (or -1 if convergence was not achieved)
    """
    x = x0.copy()             
    F_val = F(x)                
    F_norm = np.linalg.norm(F_val, ord=2)  
    steps = 0
    while F_norm > eps and steps < N:
        # Instead of using np.linalg.solve (which fails if the Jacobian is singular),
        # we use the pseudo-inverse (np.linalg.pinv) to compute an update.
        s = np.dot(np.linalg.pinv(DF(x)), F_val)
        x = x - s               
        F_val = F(x)             
        F_norm = np.linalg.norm(F_val, ord=2)  
        steps += 1
    if F_norm > eps:
        steps = -1  
    return x, steps


In [35]:
def satellite_positions_spherical(rho, phi, theta):
    """
    Given spherical coordinates:
      A = rho * cos(phi) * cos(theta)
      B = rho * cos(phi) * sin(theta)
      C = rho * sin(phi)
    Returns (A, B, C) for a single satellite.
    """
    A = rho * np.cos(phi) * np.cos(theta)
    B = rho * np.cos(phi) * np.sin(theta)
    C = rho * np.sin(phi)
    return A, B, C

# Parameters for satellite positions:
rho = 26570  # km (given)
# Choose phi values in [0, pi/2] (e.g., pi/6 for all to ensure they are in the upper hemisphere)
phi_vals = np.array([np.pi/6, np.pi/6, np.pi/6, np.pi/6])
# Choose theta values arbitrarily in [0, 2pi]: 0, pi/2, pi, 3pi/2.
theta_vals = np.array([0, np.pi/2, np.pi, 3*np.pi/2])

A = np.zeros(4)
B = np.zeros(4)
C = np.zeros(4)
for i in range(4):
    A[i], B[i], C[i] = satellite_positions_spherical(rho, phi_vals[i], theta_vals[i])


In [36]:
def F_GPS_params(x, A, B, C, t, c):
    """
    Computes the residuals for the GPS system.
    
    For each satellite i:
      F_i(x,y,z,d) = sqrt((x - A_i)^2 + (y - B_i)^2 + (z - C_i)^2) - c*(t_i - d)
    
    x: Vector [x, y, z, d]
    A, B, C: Arrays of satellite positions
    t: Array of travel times (can be baseline or perturbed)
    c: Speed of light (km/s)
    
    Returns a 4-element vector of residuals.
    """
    distances = np.sqrt((x[0] - A)**2 + (x[1] - B)**2 + (x[2] - C)**2)
    return distances - c * (t - x[3])

def DF_GPS_params(x, A, B, C, c):
    """
    Computes the 4x4 Jacobian matrix for the GPS system.
    
    For each satellite i:
      ∂F_i/∂x = (x - A_i)/r_i,
      ∂F_i/∂y = (y - B_i)/r_i,
      ∂F_i/∂z = (z - C_i)/r_i,
      ∂F_i/∂d = c,
    where r_i = sqrt((x-A_i)^2 + (x[1]-B_i)^2 + (x[2]-C_i)^2).
    """
    J = np.zeros((4, 4))
    for i in range(4):
        r_i = np.sqrt((x[0] - A[i])**2 + (x[1] - B[i])**2 + (x[2] - C[i])**2)
        J[i, 0] = (x[0] - A[i]) / r_i
        J[i, 1] = (x[1] - B[i]) / r_i
        J[i, 2] = (x[2] - C[i]) / r_i
        J[i, 3] = c
    return J

In [37]:
# Baseline receiver: (x, y, z, d) = (0, 0, 6370, 0.0001)
x_true = np.array([0.0, 0.0, 6370.0, 0.0001])
c = 299792.458  # km/s

# Compute the satellite range for each satellite:
R = np.sqrt(A**2 + B**2 + (C - 6370)**2)
# Compute baseline travel times: t = d + R/c
t_baseline = x_true[3] + R/c


In [38]:
# Use the baseline as the initial guess.
x0 = x_true.copy()

solution_baseline, steps_baseline = Newton_system(
    lambda x: F_GPS_params(x, A, B, C, t_baseline, c),
    lambda x: DF_GPS_params(x, A, B, C, c),
    x0, 100, 1e-4)  

print("Baseline solution:", solution_baseline)
print("Baseline residual:", F_GPS_params(solution_baseline, A, B, C, t_baseline, c))


Baseline solution: [0.00e+00 0.00e+00 6.37e+03 1.00e-04]
Baseline residual: [0. 0. 0. 0.]


In [39]:
# We will perturb each travel time by ±10^-8 seconds.
perturbations = [-1e-8, 1e-8]
max_pos_error = 0
max_error_magnification = 0

# Loop over all 16 combinations of perturbations for the 4 satellites.
for deltas in itertools.product(perturbations, repeat=4):
    delta_t = np.array(deltas)
    t_perturbed = t_baseline + delta_t
    solution_perturbed, steps_perturbed = Newton_system(
        lambda x: F_GPS_params(x, A, B, C, t_perturbed, c),
        lambda x: DF_GPS_params(x, A, B, C, c),
        x0, 100, 1e-4)
    # Compute the forward error: difference in (x, y, z) compared to the baseline (x_true)
    pos_error = np.linalg.norm(solution_perturbed[0:3] - x_true[0:3], ord=np.inf)
    # Compute the input error: c * ||Δt||∞ (Δt in seconds, converted to distance)
    input_error = c * np.linalg.norm(delta_t, ord=np.inf)
    error_magnification = pos_error / input_error if input_error != 0 else 0
    if pos_error > max_pos_error:
        max_pos_error = pos_error
    if error_magnification > max_error_magnification:
        max_error_magnification = error_magnification

print("Maximum position error (KM):", max_pos_error)
max_pos_error_meters = max_pos_error * 1000
print("Maximum position error (Meters):", max_pos_error_meters)
print("Estimated condition number (error magnification factor):", max_error_magnification)



Maximum position error (KM): 0.0031303716345498778
Maximum position error (Meters): 3.1303716345498778
Estimated condition number (error magnification factor): 1.0441795819125903
